<a href="https://colab.research.google.com/github/arjunkpreman/Transfer-Learning-Project/blob/main/ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# Load pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

94765736/94765736 [==============================] - 5s 0us/step


In [8]:
# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False


In [9]:
# Add new classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)  # num_classes is the number of classes in your task

In [10]:

# Create the transfer learning model
model = Model(inputs=base_model.input, outputs=predictions)

In [11]:

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
# Set up data generators for training and validation
train_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet.preprocess_input)
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Dataset/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 3015 images belonging to 10 classes.


In [13]:
val_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet.preprocess_input)
val_generator = val_datagen.flow_from_directory(
    '/content/drive/MyDrive/Dataset/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 921 images belonging to 10 classes.


In [14]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    validation_data=val_generator,
    validation_steps=val_generator.samples // 32,
    epochs=5
)

Epoch 1/5
94/94 [==============================] - 2435s 26s/step - loss: 0.6435 - accuracy: 0.7918 - val_loss: 0.3149 - val_accuracy: 0.8984
Epoch 2/5
94/94 [==============================] - 19s 206ms/step - loss: 0.1727 - accuracy: 0.9494 - val_loss: 0.2639 - val_accuracy: 0.8973
Epoch 3/5
94/94 [==============================] - 19s 197ms/step - loss: 0.1049 - accuracy: 0.9688 - val_loss: 0.2335 - val_accuracy: 0.9018
Epoch 4/5
94/94 [==============================] - 20s 208ms/step - loss: 0.0773 - accuracy: 0.9792 - val_loss: 0.2172 - val_accuracy: 0.9263
Epoch 5/5
94/94 [==============================] - 18s 195ms/step - loss: 0.0379 - accuracy: 0.9930 - val_loss: 0.2469 - val_accuracy: 0.9196


In [15]:
# Save the trained model
model.save('/content/drive/MyDrive/Dataset/ResNet50.h5')

In [16]:
model.evaluate(val_generator)

29/29 [==============================] - 5s 183ms/step - loss: 0.2412 - accuracy: 0.9218


[0.24121303856372833, 0.9218240976333618]